In [7]:
######JM Gawrych  11/30/2018 rev 3/11/2019
######B Snyder 6/26/2019 Added to IBM Watson Studio.  
#!pip install arcgis   #You must use this command the first time the notebook is run to load the arcgis module.  By using the '!' it works the same as using a command window but inside of Watson Studio.

#### Gets multipleThunderstorm Potential forecasts from severity 1 to 5.  Generates a geojson file from our featurizer with these 5 layers in one and pushes to ESRI Map in ARCGIS Online. 
#Use this link to see the results on ARCGIS Online: https://arcg.is/1zWbCW

import requests,json,datetime # need to install these libraries to run
#ESRI modules
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(44) # Radar Derived 24-Hour HailVision. 
prod_name='VAR0191209FROM25501surface12MinuteMaximum' # Per 
APIKEY='YOUR_OWN_IBM_WEATHER_KEY_HERE'  #This is IBM internal  key 


########################PRODUCT INVENTORY CALL################################  
def InvCall():
    base1 = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name
    base2 = '&apiKey='+APIKEY
    url = base1 + base2 
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json()  # Just get most current run and next forecast
        t = data["layers"][prod_num][prod_name]["dimensions"][0]["t"][31]
        rt = data["layers"][prod_num][prod_name]["dimensions"][0]["rt"][0]
        print (rt,t)
        return [t, rt]
                    
##########################   API CALL   ###################################
def APIcall(t,rt,ts): # function for pulling data 
    count=0
    threshold = 1    
    thresholdS = str(threshold)   
    num_layers = 5
    inc=1
    
    fc = {
        "type": "FeatureCollection",
        "features": []
    }
   
              
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+prod_num+':'+prod_name
    base2 = '&t='+t+'&rt='+rt+ '&apiKey='+APIKEY
    base3= '&geometryType=polygon&bbox=-160.0,10.0,-50.0,70.0' # NA bounding box

##add layer for lowest threshold and one for highest

    for x in range(0,num_layers):
        count=count+1
        base4 = '&operation=greaterThan&threshold='+thresholdS

        url = base1 + base2 + base3 + base4  # make API URL   
        print (url)  
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}
            
            f['geometry'] = y['geometry']
            f['properties']['tStormIndex'] = y['properties']['threshold']
            f['properties']['Date'] = ts
            fc['features'].append(f)           
        
        threshold=threshold+inc
        thresholdS=str(threshold)                  
    
    with open("TstormIndex.geojson", "w") as outfile:
        json.dump(fc, outfile)
    
################################################################################
values = InvCall()
t= values[0]
rt=values[1]

now=datetime.datetime.utcnow()
year=str(now.year)
month=str('{:02d}'.format(now.month))
day=str('{:02d}'.format(now.day))
hour=str('{:02d}'.format(now.hour))
ts=year+"-"+month+"-"+day+","+hour+"00"

APIcall(t,rt,ts)

################################################################################
##  ADD CONTENT TO ARCGIS  
# Connect to the GIS Using Your Credentials
portalURL = "http://ibmmain.maps.arcgis.com"
userID = "ESRI_USERNAME"
password = "ESRI_PASSWORD"
gis = GIS(portalURL, userID, password)

#Update each piece of content
existing_ID = 'ecc1b86e1a264a65a46b99e39aaac3c7'  # Map ID
wx_item = gis.content.get(existing_ID)
collection = features.FeatureLayerCollection.fromitem(wx_item)
collection.manager.overwrite('TstormIndex.geojson')  
print ("Updating Layer")

1561834800000 1561852800000
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1561852800000&rt=1561834800000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&bbox=-160.0,10.0,-50.0,70.0&operation=greaterThan&threshold=1
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1561852800000&rt=1561834800000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&bbox=-160.0,10.0,-50.0,70.0&operation=greaterThan&threshold=2
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1561852800000&rt=1561834800000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&bbox=-160.0,10.0,-50.0,70.0&operation=greaterThan&threshold=3
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1561852800000&rt=1561834800000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&bbox=-160.0,10